In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import io

# Display a message to the user
display(HTML("<h3 style='color:teal;'>📂 Please upload a CSV file to begin:</h3>"))

# File upload widget
upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='📁 Upload CSV ...',
    style={'button_color': 'lightgreen'}
)

# Output area
output = widgets.Output()

# Upload handler
def handle_upload(change):
    global df
    with output:
        clear_output()
        if upload_widget.value:
            uploaded_file = upload_widget.value[0]  # get the first uploaded file
            content = uploaded_file['content']
            df = pd.read_csv(io.BytesIO(content))
            print("✅ CSV successfully loaded!")
            print(f"Shape: {df.shape}")
            display(df.head())

# Attach handler
upload_widget.observe(handle_upload, names='value')

# Show interface
display(upload_widget, output)


FileUpload(value=(), accept='.csv', description='📁 Upload CSV ...', style=ButtonStyle(button_color='lightgreen…

Output()

In [3]:
import pandas as pd
from pyvis.network import Network

# Read the CSV file (make sure the CSV file is in the same folder)
#df = pd.read_csv("/Users/rohanpersonal/git_projs/argument_mining/latest_mappings/Mapping MGRs (v4 - JSV edits) - Nagoya_OK_JSV rev 2 May.csv")

# Initialize the Pyvis network: directed graph, with a fixed height/width.
net = Network(height="800px", width="100%", directed=True)

# Collect unique actor nodes and accumulate action nodes with their provisions.
actor_nodes = set()         # Store unique actor names.
action_nodes = {}           # Dictionary: key = action text, value = list of provisions.
edges = []                  # List of tuples: (source, target, label).

# Process each row in the CSV.
for index, row in df.iterrows():
    provision = str(row['Provision'])
    actor_holder = row['Actor‑Holder']
    action = row['Action']
    actor_target = row['Actor‑Affected (Target)']
    
    # Collect actor nodes (without accumulating provisions).
    actor_nodes.add(actor_holder)
    actor_nodes.add(actor_target)
    
    # Accumulate provisions for each action.
    if action in action_nodes:
        if provision not in action_nodes[action]:
            action_nodes[action].append(provision)
    else:
        action_nodes[action] = [provision]
    
    # Add edges: from Actor‑Holder to Action with label "Has"
    edges.append((actor_holder, action, "Has"))
    # And from Action to Actor‑Affected (Target) with label "Towards"
    edges.append((action, actor_target, "Towards"))

# Add actor nodes without any provision tooltip.
for actor in actor_nodes:
    net.add_node(actor, 
                 label=actor, 
                 shape="ellipse", 
                 color="blue", 
                 group="actor", 
                 font={"size": 10},
                 size=20,
                 widthConstraint={"maximum": 100})

# Add action nodes with tooltips showing the associated article numbers.
# Here we reduce the box size and font size for action nodes.
for action, provisions in action_nodes.items():
    title_text = "Provisions: " + ", ".join(provisions)
    net.add_node(action, 
                 label=action, 
                 title=title_text,
                 shape="box", 
                 color="yellow", 
                 group="action", 
                 font={"size": 10},    # reduced text size (from 12 to 10)
                 size=20,              # reduced overall size (from 30 to 20)
                 widthConstraint={"maximum": 100})  # reduced width (from 150 to 100)

# Add the edges (all edges are black and have arrow heads).
for src, dst, label in edges:
    net.add_edge(src, dst, label=label, arrows="to", color="black")

# Add the physics control buttons so that users can adjust gravity and other settings.
net.show_buttons(filter_=["physics"])

# Generate the HTML code from the Pyvis network.
html_str = net.generate_html()

# --- Inject custom CSS, search box and live search functionality ---
# The CSS below styles the physics control panel (with id "mynetwork-popUp") so that it appears fixed at the bottom-right,
# is small, and does not occupy much space.
# The JavaScript stores each node's original color so that when a user types in the search box,
# only matching nodes are highlighted in pink and non-matching nodes retain their original color.
injection = """
<style>
  /* Custom style for the physics control panel */
  #mynetwork-popUp {
      position: fixed;
      bottom: 10px;
      right: 10px;
      top: auto !important;
      width: auto;
      height: auto;
      padding: 5px;
      background: rgba(255,255,255,0.7);
      border: 1px solid #ccc;
      border-radius: 5px;
      z-index: 500;
  }
  #mynetwork-popUp .vis-button {
      font-size: 10px;
      padding: 2px 4px;
  }
</style>
<div style="position: absolute; top: 10px; right: 10px; z-index: 1000;">
  <input type="text" id="searchBox" placeholder="Search..." style="padding: 5px;">
</div>
<script type="text/javascript">
  // Store original colors for all nodes.
  var originalNodeColors = {};
  network.body.data.nodes.get().forEach(function(node) {
      originalNodeColors[node.id] = node.color;
  });
  
  document.getElementById("searchBox").addEventListener("keyup", function() {
      var searchText = this.value.toLowerCase();
      network.body.data.nodes.get().forEach(function(node) {
          if(searchText === ""){
              // Reset to original color when search box is cleared.
              network.body.data.nodes.update({id: node.id, color: originalNodeColors[node.id]});
          } else {
              var labelMatch = node.label.toLowerCase().includes(searchText);
              var titleMatch = (node.title && node.title.toLowerCase().includes(searchText));
              if(labelMatch || titleMatch) {
                  network.body.data.nodes.update({id: node.id, color: 'pink'});
              } else {
                  // Keep original color for non-matching nodes.
                  network.body.data.nodes.update({id: node.id, color: originalNodeColors[node.id]});
              }
          }
      });
  });
</script>
"""

# Insert the search box, style, and script just before the closing </body> tag.
html_str = html_str.replace("</body>", injection + "\n</body>")



# Save the HTML file in the current directory with a clean name
output_filename = "nagoya_network_v4.html"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(html_str)

print(f"✅ Network graph saved as '{output_filename}' in the current directory.")


# Write the resulting HTML to the output file "gratk_network.html"
#with open("/Users/rohanpersonal/git_projs/argument_mining/latest_mappings/gratk_network_v4_2nd.html", "w", encoding="utf-8") as f:
#    f.write(html_str)




✅ Network graph saved as 'nagoya_network_v4_11may.html' in the current directory.
